# Import

In [1]:
import re
import requests
import pandas        as pd

from bs4             import BeautifulSoup
from datetime        import datetime

# API building

In [ ]:
#Link do acesso
url = requests.get("https://www.autoscout24.com/lst/bmw?atype=C&cy=I&desc=0&fregfrom=2018&fregto=2022&powertype=kw&search_id=2ajkso6erxn&sort=standard&source=detailsearch&ustate=N%2CU")

#Salvando o objeto
content = url.content
url_final = []

#Convertendo para um objeto do tipo BeautifulSoup
soup = BeautifulSoup(content, 'html.parser')

list_auto = []

# Acess all products

In [3]:
def gerar_links_base(prefixo, sufixo, num_iteracoes, final_url):
    links = []

    for i in range(1, num_iteracoes + 1):
        link = f"{prefixo}{i}{sufixo}{final_url}"
        links.append(link)

    return links

# construindo o link:
prefixo = "https://www.autoscout24.com/lst/bmw?atype=C&cy=I&desc=0&fregfrom=2018&fregto=2022&page="

sufixo = ""
num_iteracoes = 20

final_url = "&powertype=kw&search_id=2ajkso6erxn&sort=standard&source=listpage_pagination&ustate=N%2CU"

links_gerados = gerar_links_base(prefixo, sufixo, num_iteracoes, final_url)
print(links_gerados)

['https://www.autoscout24.com/lst/bmw?atype=C&cy=I&desc=0&fregfrom=2018&fregto=2022&page=1&powertype=kw&search_id=2ajkso6erxn&sort=standard&source=listpage_pagination&ustate=N%2CU', 'https://www.autoscout24.com/lst/bmw?atype=C&cy=I&desc=0&fregfrom=2018&fregto=2022&page=2&powertype=kw&search_id=2ajkso6erxn&sort=standard&source=listpage_pagination&ustate=N%2CU', 'https://www.autoscout24.com/lst/bmw?atype=C&cy=I&desc=0&fregfrom=2018&fregto=2022&page=3&powertype=kw&search_id=2ajkso6erxn&sort=standard&source=listpage_pagination&ustate=N%2CU', 'https://www.autoscout24.com/lst/bmw?atype=C&cy=I&desc=0&fregfrom=2018&fregto=2022&page=4&powertype=kw&search_id=2ajkso6erxn&sort=standard&source=listpage_pagination&ustate=N%2CU', 'https://www.autoscout24.com/lst/bmw?atype=C&cy=I&desc=0&fregfrom=2018&fregto=2022&page=5&powertype=kw&search_id=2ajkso6erxn&sort=standard&source=listpage_pagination&ustate=N%2CU', 'https://www.autoscout24.com/lst/bmw?atype=C&cy=I&desc=0&fregfrom=2018&fregto=2022&page=6&powe

## Building Table Initial


In [4]:
# Access the info on the site and building the table
list_auto = []
for link in links_gerados:
    #print(link)
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    page = requests.get( link )

    #Salvando o objeto
    content = page.content
    #url_final = []
    
    #Convertendo para um objeto do tipo BeautifulSoup
    soup = BeautifulSoup(content, 'html.parser')

    # Beautiful Soup object
    soup = BeautifulSoup( page.text, 'html.parser' )

    add = soup.find_all('article', re.compile('cldt-summary-full-item'))
     #Seller/Location
    seller_detail = soup.find('div', attrs={'class':re.compile('SellerInfo')})
    seller = seller_detail.find('span', attrs={'class':re.compile('SellerInfo_name')}).get_text().strip()
    endereco = seller_detail.find('span', attrs={'class':re.compile('SellerInfo_address')}).get_text().strip()
        
      
    for ads in add:
        #auto = soup.find('a', attrs={'class':re.compile('ListItem_title')}).get_text().strip()
        id_ads = ads['id']
        manufacturer_by = ads['data-make']
        model = ads['data-model']
        km = ads['data-mileage']
        year = ads['data-first-registration']
        price = ads['data-price']
        version = ads.find('span', attrs={'class': re.compile("ListItem_version")}).getText()
        description_car = ads.find('div', attrs={'class': re.compile("ListItem_wrapper")}).getText()
        
        list_auto.append([id_ads, manufacturer_by, model, version, km, year, price, seller, endereco])
        
        df = pd.DataFrame(list_auto, columns=['id_ads', 'manufacturer_by', 'model', 'version',
                                              'km', 'year', 'price', 'seller',
                                              'address']).reset_index(drop=True)
        df['scrapy_datetime'] = datetime.now().strftime('%Y-%m-%d')


        #new_df = pd.DataFrame(df)
        
        # Carregue a tabela já existente, se houver
        try:
            tabela_existente = pd.read_csv('../dataset/list_auto.csv', encoding='utf-8', sep=';')
        except FileNotFoundError:
            # Se a tabela não existe, crie um DataFrame vazio
            tabela_existente = pd.DataFrame()
        
        # Concatening new DataFrame with exist table
        df_new = pd.concat([tabela_existente, df], ignore_index=False)

# Save the concatenated table on the new file.
df_new.to_csv('../dataset/list_auto.csv', encoding='utf-8', sep=';' ,index=False) 
        
rows = df_new.shape[0]
print(rows)


# Save the concatenated table on the new file.
#df_new.to_csv('../dataset/list_auto.csv', encoding='utf-8', sep=';' ,index=False) 

AttributeError: 'NoneType' object has no attribute 'find'

# Load DataSet

In [5]:
df_scout_dirty = pd.read_csv('../dataset/list_auto.csv', encoding='utf-8', sep=';')


In [6]:
df_scout_dirty.head()

,id_ads,manufacturer_by,model,version,km,year,price,seller,endereco,scrapy_datetime
0,8c9aef9c-1513-41ab-8db4-10686decdfb5,bmw,116,d 5p Msport auto,146000,06-2019,16500,Bryan Auto Srl,BRYAN AUTO • IT-13100 Vercelli,2024-11-22
1,78b02ce1-2c8f-4caf-88fc-7d59b6e42cda,bmw,x1,xdrive18d Business auto my18,100000,09-2018,18999,Bryan Auto Srl,BRYAN AUTO • IT-13100 Vercelli,2024-11-22
2,c3eb9e5d-f4e3-4864-a283-32063782cd8b,bmw,318,d Touring MHEV Business Advantage 150CV auto,140000,11-2020,19490,Bryan Auto Srl,BRYAN AUTO • IT-13100 Vercelli,2024-11-22
3,a564e061-226c-464f-bf1a-b178e7b929dc,bmw,116,116d 5p. Msport,53050,02-2019,19900,Bryan Auto Srl,BRYAN AUTO • IT-13100 Vercelli,2024-11-22
4,6e6ae0b6-d199-4d2e-96ec-d525bfb48e1e,bmw,730,d xDrive Eccelsa,226135,02-2018,25900,Bryan Auto Srl,BRYAN AUTO • IT-13100 Vercelli,2024-11-22
